# Connect to Snowflake using `Python3`


#### Topics covered in this example
* Installing Snowflake connector for Python
* Connecting to Snowflake using `Python3`

## Table of Contents:

1. [Prerequisites](#Prerequisites)
2. [Introduction](#Introduction)
3. [Install dependency libraries](#Install-dependency-libraries)
4. [Connect to Snowflake using `Python3`](#Connect-to-Snowflake)
   * [Read Data from Snowflake table](#Read-data-from-Snowflake-table-using-Python3)
   * [Create Pandas DataFrame from Snowflake table](#Create-Pandas-DataFrame-from-Snowflake-table-using-Python3)
   * [Write Data to Snowflake table](#Write-data-to-Snowflake-table-using-Python3)
5. [Close connection](#Close-connection-and-cleanup)

***

## Prerequisites
<div class="alert alert-block alert-info">
<b>NOTE :</b> In order to execute this notebook successfully as is, please ensure the following prerequisites are completed.</div>

* This notebook is tested with Amazon EMR 6.4.0
* This example installs the Snowflake connector and other dependant Python packages using pip, hence the EMR cluster attached to this notebook must have internet connectivity. In case using custom package repository for pip, please make sure you have these dependant packages hosted in your custom repository.
***

## Introduction
In this example we use `Python3` to connect to a table in Snowflake using the Snowflake connector for Python.

The Snowflake Connector for Python provides an interface for developing Python applications that can connect to Snowflake and perform all standard operations.

The connector is a native, pure Python package that has no dependencies on JDBC or ODBC. It can be installed using `pip` on Linux, macOS, and Windows platforms where Python 3.6, 3.7, 3.8, or 3.9 is installed.

For more information please find the [documentation here](#https://docs.snowflake.com/en/user-guide/python-connector.html)
***

## Install dependency libraries

- We will need a few Python packages for Snowflake connector for Python to work. The Snowflake connector for Python has a list of required packages specific for Python version before we can install `snowflake-connector-python`. In this example, we are using Python 3.7 and testing `snowflake-connector-python` version 2.6.2.

- Once we install the required dependencies, we will install below additional Python packages:

  * `snowflake-connector-python`
  * `snowflake-connector-python[pandas]`
  * `PyArrow`
  
<div class="alert alert-block alert-info">
<b>NOTE :</b> Please note that you may need to restart the kernel after install all the dependant Python package.</div>

In [ ]:
%pip install -r https://raw.githubusercontent.com/snowflakedb/snowflake-connector-python/v2.6.2/tested_requirements/requirements_37.reqs

In [ ]:
%pip install snowflake-connector-python==2.6.2 snowflake-connector-python[pandas]==2.6.2 pyarrow

## Connect to Snowflake

For this example we will connect to Snowflake account `abc12345` hosted in the AWS region `us-east-1`. The connection will be established using user `SNOWFLAKE_USER` with the password `My_Password`.

Please make sure you replace these values with the ones appropriate for your environment/setup.

Lets create a connection context which will be used in examples below:

In [12]:
import snowflake.connector
import getpass

ctx = snowflake.connector.connect(
    user='SNOWFLAKE_USER',
    password=getpass.getpass('Enter Snowflake user password'),
    account='abc12345.us-east-1'
    )

print('Created a connection context: ' +str(ctx))

Enter Snowflake user password ··········


Created a connection context: <snowflake.connector.connection.SnowflakeConnection object at 0x7fac5c5edf10>


### Read data from Snowflake table using `Python3`

We see a simple example of connecting to Snowflake table to fetch 5 rows (using `fetchmany()`. You can also use `fetchall()` or `fetchone()`

We will connect to table `CUSTOMER` within schema `TPCH_SF1` present inside the database `SNOWFLAKE_SAMPLE_DATA` using the connect context `ctx` created above.

Please make sure you replace these values with the ones appropriate for your environment/setup.

The following example will return the values/rows in Python list `result`

In [ ]:
query = "SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER"

cs = ctx.cursor()
try:
    cs.execute(query)
    result = cs.fetchmany(5)
    print(result)
finally:
    cs.close()

### Create Pandas DataFrame from Snowflake table using `Python3`

We will now see an example of creating a Pandas DataFrame.

We will connect to table `CUSTOMER` within schema `TPCH_SF1` present inside the database `SNOWFLAKE_SAMPLE_DATA` using the connect context `ctx` created above.

Please make sure you replace these values with the ones appropriate for your environment/setup.

In [ ]:
import pandas as pd

query = "SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCH_SF1.CUSTOMER"

cs = ctx.cursor()
try:
    cs.execute(query)
    pandasTable = cs.fetch_pandas_all()
    # Let's validate the data type for pandasTable
    print('The data type returned is: ' + str(type(pandasTable)))
    
    # Check the values returned
    print(pandasTable)
finally:
    cs.close()

### Write data to Snowflake table using `Python3`

We take a few examples to see how we can create a table and insert data into this table.

We will create a sample table named `SAMPLETABLE` inside the same

In [ ]:
cs = ctx.cursor()
try:
    # Create database SAMPLEDB
    cs.execute("CREATE DATABASE IF NOT EXISTS SAMPLEDB")
    
    # Create schema myschema inside sampledb database
    cs.execute("CREATE SCHEMA SAMPLEDB.MYSCHEMA")
    
    # Create table SAMPLETABLE
    cs.execute("CREATE OR REPLACE TABLE SAMPLEDB.MYSCHEMA.SAMPLETABLE(id integer, name string)")
    
    # Insert values inside SAMPLETABLE
    cs.execute("INSERT INTO SAMPLEDB.MYSCHEMA.SAMPLETABLE VALUES (1, 'john jones'), (2, 'tracey smith'), (3, 'amy sanders')")
finally:
    cs.close()

You can also use bulk insert to load the data into the Snowflake table instead of using single-row INSERT SQL statements. The bulk load can be performed from an internal (local) storage or from an external storage like Amazon S3 bucket. The following example assumes that you have the data file located on S3 bucket `MYDATABUCKET`.

Please note that this example uses storage integration instead of supplying access credentials in the execute statment. We recommend using storage integration which should be created before running the load statements. For more details on creating storage integration, please refer to the [documentation here](#https://docs.snowflake.com/en/sql-reference/sql/create-storage-integration.html)

In [ ]:
cs = ctx.cursor()
try:
    cs.execute("""
    COPY INTO SAMPLEDB.MYSCHEMA.SAMPLETABLE FROM s3://MYDATABUCKET/data-files/
        STORAGE_INTEGRATION = S3DataBucket
        FILE_FORMAT=(field_delimiter=',')
    """)
finally:
    cs.close()

## Close connection and cleanup

Finally, we close the connection context

In [ ]:
ctx.close()